# Fine-tuning XLM-R on CodiEsp-P

In this notebook, following a multi-label sequence classification approach, the XLM-R model is fine-tuned on both the training and development sets of the CodiEsp-P corpus. Additionally, the predictions made by the model on the test set are saved, in order to futher evaluate the clinical coding performance of the model (see `results/CodiEsp-P/Evaluation.ipynb`).

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

# Auxiliary components
import sys
sys.path.append("..")
from nlp_utils import *

# XLM-R tokenizer
from transformers import XLMRobertaTokenizer
import sentencepiece_pb2
model_name = "xlm-roberta-base"
tokenizer = XLMRobertaTokenizer.from_pretrained(model_name)
spt = sentencepiece_pb2.SentencePieceText()

# Hyper-parameters
text_col = "raw_text"
SEQ_LEN = 128
BATCH_SIZE = 16
EPOCHS = 41
LR = 3e-5

random_seed = 0
tf.random.set_seed(random_seed)

## Load text

Firstly, all text files from training and development CodiEsp corpora are loaded in different dataframes.

Also, CIE-Procedimiento codes are loaded.

In [2]:
corpus_path = "../datasets/codiesp_v4/"

### Training corpus

In [3]:
%%time
train_path = corpus_path + "train/text_files/"
train_files = [f for f in os.listdir(train_path) if os.path.isfile(train_path + f)]
train_data = load_text_files(train_files, train_path)
df_text_train = pd.DataFrame({'doc_id': [s.split('.txt')[0] for s in train_files], 'raw_text': train_data})

CPU times: user 6.23 ms, sys: 4.13 ms, total: 10.4 ms
Wall time: 9.81 ms


In [4]:
df_text_train.shape

(500, 2)

In [5]:
df_text_train.head()

,doc_id,raw_text
0,S0004-06142007000600016-2,Paciente varón de 35 años con tumoración en po...
1,S1137-66272009000500017-1,Lactante de sexo femenino que ingresó a los 7 ...
2,S0365-66912007001100010-1,Paciente de 63 años que refería déficit de agu...
3,S0365-66912009000300010-1,Se presenta el caso de un varón de 24 años de ...
4,S0211-69952013000500035-1,Se presenta el caso de un varón de 64 años sin...


In [6]:
df_text_train.raw_text[0]

'Paciente varón de 35 años con tumoración en polo superior de teste derecho hallada de manera casual durante una autoexploración, motivo por el cual acude a consulta de urología donde se realiza exploración física, apreciando masa de 1cm aproximado de diámetro dependiente de epidídimo, y ecografía testicular, que se informa como lesión nodular sólida en cabeza de epidídimo derecho. Se realiza RMN. Confirmando masa nodular, siendo el tumor adenomatoide de epidídimo la primera posibilidad diagnóstica.\n\nSe decide, en los dos casos, resección quirúrgica de tumoración nodular en cola epidídimo derecho, sin realización de orquiectomía posterior.\nEn ambos casos se realizó examen anátomopatológico de la pieza quirúrgica. Hallazgos histológicos macroscópicos: formación nodular de 1,5 cms (caso1) y 1,2 cms (caso 2) de consistencia firme, coloración blanquecina y bien delimitada. Microscópicamente se observa proliferación tumoral constituida por estructuras tubulares en las que la celularidad 

We also load the CIE-Procedimiento codes table:

In [7]:
df_codes_train = pd.read_table(corpus_path + "train/trainP.tsv", sep='\t', header=None)

In [8]:
df_codes_train.columns = ["doc_id", "code"]

In [9]:
df_codes_train.shape

(1550, 2)

In [10]:
df_codes_train.head()

,doc_id,code
0,S0004-06142005000700014-1,bw03zzz
1,S0004-06142005000700014-1,3e02329
2,S0004-06142005000700014-1,bw40zzz
3,S0004-06142005000700014-1,bv44zzz
4,S0004-06142005000700014-1,bn20


In [11]:
len(set(df_codes_train["doc_id"]))

435

### Development corpus

In [12]:
%%time
dev_path = corpus_path + "dev/text_files/"
dev_files = [f for f in os.listdir(dev_path) if os.path.isfile(dev_path + f)]
dev_data = load_text_files(dev_files, dev_path)
df_text_dev = pd.DataFrame({'doc_id': [s.split('.txt')[0] for s in dev_files], 'raw_text': dev_data})

CPU times: user 22.3 ms, sys: 0 ns, total: 22.3 ms
Wall time: 22.2 ms


In [13]:
df_text_dev.shape

(250, 2)

In [14]:
df_text_dev.head()

,doc_id,raw_text
0,S1698-44472004000100009-1,Varón de 64 años de edad con tumefacción mandi...
1,S1139-76322015000300013-1,Niña de tres años que acude a Urgencias tras l...
2,S1130-05582015000100004-1,Se presenta el caso de una mujer de 60 años de...
3,S1887-85712015000200005-1,Paciente varón de cinco años de edad que tras ...
4,S1699-65852010000300002-1,"LTR. Paciente de sexo masculino, de 32 años de..."


In [15]:
df_text_dev.raw_text[0]

'Varón de 64 años de edad con tumefacción mandibular derecha de 6 meses de evolución. La radiografía simple mostraba una lesión expansiva bien delimitada, osteolítica, multiloculada, localizada en rama horizontal mandibular. La tomografía computerizada presentaba una lesión expansiva con destrucción de la cortical ósea. Con el diagnóstico provisional de probable ameloblastoma se procedió a la resección-biopsia de la lesión. Mediante incisión interpapilar se expuso la mandíbula que mostraba la superficie abombada y destruída por una tumoración carnosa de consistencia densa que rodeaba la rama del nervio dentario inferior. Tras un cuidadoso curetaje de la cavidad ósea se reconstruyó la mandíbula y se repuso la mucosa. No hubo complicaciones postquirúrgicas.\n\nEl material remitido a Anatomía Patológica consistía en fragmentos tumorales de unos 2x1.5 cm, blanco-grisáceos al corte y de consistencia firme. Se tomaron diversas muestras que tras fijarse en formaldehído se incluyeron en parafi

We also load the CIE-Procedimiento codes table:

In [16]:
df_codes_dev = pd.read_table(corpus_path + "dev/devP.tsv", sep='\t', header=None)

In [17]:
df_codes_dev.columns = ["doc_id", "code"]

In [18]:
df_codes_dev.shape

(817, 2)

In [19]:
df_codes_dev.head()

,doc_id,code
0,S0004-06142005000900016-1,bt41zzz
1,S0004-06142005000900016-1,ct13
2,S0004-06142005001000011-1,3e1m39z
3,S0004-06142005001000011-1,0tcb
4,S0004-06142005001000011-1,bt02


In [20]:
len(set(df_codes_dev["doc_id"]))

222

We join the training and development CodiEsp codes dataframes together:

In [21]:
df_codes_train_dev = pd.concat([df_codes_train, df_codes_dev])

In [22]:
df_codes_train_dev.shape

(2367, 2)

In [23]:
df_codes_train_dev.head()

,doc_id,code
0,S0004-06142005000700014-1,bw03zzz
1,S0004-06142005000700014-1,3e02329
2,S0004-06142005000700014-1,bw40zzz
3,S0004-06142005000700014-1,bv44zzz
4,S0004-06142005000700014-1,bn20


## Creating corpora of annotated sentences

Leveraging the information available for the named-entity-recognition and normalization (NER-N) CodiEsp-X task, we create both a training and a development corpus of annotated sentences with CIE-Procedimiento codes.

Firstly, we pre-process the NER-N precedure-codes annotations available for both the training and development corpora.

In [24]:
# Training corpus

In [25]:
%%time

codiesp_x_train = pd.read_table(corpus_path + "train/trainX.tsv", sep='\t', header=None)

CPU times: user 35.9 ms, sys: 3.71 ms, total: 39.6 ms
Wall time: 38.8 ms


In [26]:
codiesp_x_train.columns = ["doc_id", "type", "code", "word", "location"]

In [27]:
codiesp_x_train.shape

(9181, 5)

In [28]:
codiesp_x_train.head()

,doc_id,type,code,word,location
0,S0004-06142005000700014-1,PROCEDIMIENTO,bw03zzz,Rx tórax,2163 2171
1,S0004-06142005000700014-1,PROCEDIMIENTO,3e02329,Estreptomicina intramuscular,2787 2801;2810 2823
2,S0004-06142005000700014-1,DIAGNOSTICO,n44.8,teste derecho aumentado de tamaño,1343 1376
3,S0004-06142005000700014-1,DIAGNOSTICO,z20.818,exposición a Brucella,594 615
4,S0004-06142005000700014-1,DIAGNOSTICO,r60.9,edemas,1250 1256


In [29]:
codiesp_x_train = codiesp_x_train[codiesp_x_train["type"] == "PROCEDIMIENTO"]

In [30]:
codiesp_x_train.shape

(1972, 5)

In [31]:
df_codes_train_ner = process_ner_labels(codiesp_x_train).sort_values(["doc_id", "start", "end"])

In [32]:
df_codes_train_ner.head()

,doc_id,type,code,word,start,end
0,S0004-06142005000700014-1,PROCEDIMIENTO,bw03zzz,Rx tórax,2163,2171
3,S0004-06142005000700014-1,PROCEDIMIENTO,bw40zzz,Ecografía abdominal,2173,2192
5,S0004-06142005000700014-1,PROCEDIMIENTO,bn20,TAC craneal,2194,2205
4,S0004-06142005000700014-1,PROCEDIMIENTO,bv44zzz,Ecografía testicular,2287,2307
1,S0004-06142005000700014-1,PROCEDIMIENTO,3e02329,Estreptomicina intramuscular,2787,2801


In [33]:
df_codes_train_ner.shape

(2769, 6)

In [34]:
# Development corpus

In [35]:
%%time

codiesp_x_dev = pd.read_table(corpus_path + "dev/devX.tsv", sep='\t', header=None)

CPU times: user 23.9 ms, sys: 0 ns, total: 23.9 ms
Wall time: 23.2 ms


In [36]:
codiesp_x_dev.columns = ["doc_id", "type", "code", "word", "location"]

In [37]:
codiesp_x_dev.shape

(4477, 5)

In [38]:
codiesp_x_dev.head()

,doc_id,type,code,word,location
0,S0004-06142005000900016-1,PROCEDIMIENTO,bt41zzz,ecografía renal derecha,307 316;348 361
1,S0004-06142005000900016-1,PROCEDIMIENTO,ct13,gammagrafía renal,739 756
2,S0004-06142005000900016-1,DIAGNOSTICO,q62.11,estenosis en la unión pieloureteral derecha,540 583
3,S0004-06142005000900016-1,DIAGNOSTICO,n28.89,ectasia pielocalicial,326 347
4,S0004-06142005000900016-1,DIAGNOSTICO,n39.0,infecciones del tracto urinario,198 229


In [39]:
codiesp_x_dev = codiesp_x_dev[codiesp_x_dev["type"] == "PROCEDIMIENTO"]

In [40]:
codiesp_x_dev.shape

(1046, 5)

In [41]:
df_codes_dev_ner = process_ner_labels(codiesp_x_dev).sort_values(["doc_id", "start", "end"])

In [42]:
df_codes_dev_ner.head()

,doc_id,type,code,word,start,end
0,S0004-06142005000900016-1,PROCEDIMIENTO,bt41zzz,ecografía renal derecha,307,316
1,S0004-06142005000900016-1,PROCEDIMIENTO,bt41zzz,ecografía renal derecha,348,361
2,S0004-06142005000900016-1,PROCEDIMIENTO,ct13,gammagrafía renal,739,756
3,S0004-06142005001000011-1,PROCEDIMIENTO,3e1m39z,diálisis peritoneal,95,114
7,S0004-06142005001000011-1,PROCEDIMIENTO,0270,angioplastia transluminal de la coronaria derecha,424,473


In [43]:
df_codes_dev_ner.shape

(1540, 6)

Now, using the character start-end positions of each sentence from the CodiEsp corpus (see `datasets/CodiEsp-Sentence-Split.ipynb`), we annotate the sentences with CIE-Procedimiento codes. Also, using XLM-R tokenizer, each sentence is converted into a sequence of subwords, which are further converted into vocabulary indices (input IDs) and attention mask arrays (XLM-R input tensors). We also generate a *fragments* dataset indicating the number of produced annotated sentences for each document.

In [44]:
# Sentence-Split information
ss_corpus_path = "../datasets/CodiEsp-SSplit-text_SE_All_Pos/"

### Training corpus

In [45]:
label_list = list(df_codes_train_dev["code"])

In [46]:
len(label_list)

2367

In [47]:
len(set(label_list))

727

In [48]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb_encoder = MultiLabelBinarizer()
mlb_encoder.fit([label_list])

MultiLabelBinarizer()

In [49]:
# Number of distinct codes
num_labels = len(mlb_encoder.classes_)

In [50]:
num_labels

727

Only training texts that are annotated with CIE-Procedimiento codes are considered:

In [51]:
# Some train documents (texts) are not annotated 
len(set(df_text_train["doc_id"]) - set(df_codes_train_ner["doc_id"]))

65

In [52]:
train_doc_list = sorted(set(df_codes_train_ner["doc_id"]))

In [53]:
len(train_doc_list)

435

In [54]:
# Sentence-Split data

In [55]:
%%time
ss_sub_corpus_path = ss_corpus_path + "train/"
ss_files = [f for f in os.listdir(ss_sub_corpus_path) if os.path.isfile(ss_sub_corpus_path + f)]
ss_dict_train = load_ss_files(ss_files, ss_sub_corpus_path)

CPU times: user 10.5 ms, sys: 4.22 ms, total: 14.7 ms
Wall time: 14.6 ms


In [56]:
%%time
train_ind, train_att, train_y, train_frag, train_start_end_frag = ss_create_frag_input_data_xlmr(df_text=df_text_train, 
                                                  text_col=text_col, 
                                                  df_ann=df_codes_train_ner, doc_list=train_doc_list, ss_dict=ss_dict_train,
                                                  tokenizer=tokenizer, sp_pb2=spt, lab_encoder=mlb_encoder, seq_len=SEQ_LEN)

100%|██████████| 435/435 [00:04<00:00, 97.86it/s] 


CPU times: user 4.56 s, sys: 14.6 ms, total: 4.58 s
Wall time: 4.55 s


In [57]:
# Sanity check

In [58]:
train_ind.shape

(7013, 128)

In [59]:
train_att.shape

(7013, 128)

In [60]:
train_y.shape

(7013, 727)

In [61]:
len(train_frag)

435

In [62]:
len(train_start_end_frag)

7013

In [63]:
# Check n_frag distribution across texts
pd.Series(train_frag).describe()

count    435.000000
mean      16.121839
std        7.762687
min        4.000000
25%       10.500000
50%       15.000000
75%       20.000000
max       54.000000
dtype: float64

In [64]:
# Inspect a randomly selected text and its encoded version
check_id = np.random.randint(low=0, high=len(train_doc_list), size=1)[0]

In [65]:
check_id

241

In [66]:
train_doc_list[check_id]

'S1130-01082007001100009-1'

In [67]:
df_text_train[df_text_train["doc_id"] == train_doc_list[check_id]][text_col].values[0]

'Mujer de 42 años estudiada en Consultas de Gastroenterología desde enero de 2005 por presentar rectorragia escasa al final de la deposición junto con tenesmo y dolor anal de dos años de evolución. La paciente no tenía ningún antecedente familiar ni personal de interés ni seguía ningún tratamiento médico de forma habitual. En la exploración física tan sólo se objetivó en el tacto rectal una zona indurada en la pared lateral izquierda del recto. Se realizó una analítica completa que resultó rigurosamente normal. Ante este hallazgo se decidió realizar una colonoscopia encontrándose dos pólipos milimétricos en ampolla rectal, hemorroides internas (una de ellas prolapsada) y en la cara lateral izquierda de la ampolla rectal un área eritematosa y levemente sobreelevada, de aproximadamente 2 x 2 cm, de bordes geográficos y con un aspecto indeterminado de la que se tomaron biopsias. Se planteó inicialmente el diagnóstico diferencial entre enfermedad inflamatoria intestinal (colitis ulcerosa),

In [68]:
check_id_frag = sum(train_frag[:check_id])
for i in range(check_id_frag, check_id_frag + train_frag[check_id]):
    print(mlb_encoder.inverse_transform(np.array([train_y[i]])), "\n")

[()] 

[()] 

[('0djdxzz',)] 

[()] 

[('0djd8zz',)] 

[()] 

[()] 

[()] 



In [69]:
for i in range(check_id_frag, check_id_frag + train_frag[check_id]):
    print(list(zip([tokenizer._convert_id_to_token(int(ind)) for ind in train_ind[i]][1:len(train_start_end_frag[i])+1], 
               train_start_end_frag[i])))
    print("\n")

[('▁Mu', (0, 2)), ('jer', (2, 5)), ('▁de', (5, 8)), ('▁42', (8, 11)), ('▁años', (11, 17)), ('▁estudia', (17, 25)), ('da', (25, 27)), ('▁en', (27, 30)), ('▁Consulta', (30, 39)), ('s', (39, 40)), ('▁de', (40, 43)), ('▁Gastro', (43, 50)), ('enter', (50, 55)), ('ología', (55, 62)), ('▁desde', (62, 68)), ('▁enero', (68, 74)), ('▁de', (74, 77)), ('▁2005', (77, 82)), ('▁por', (82, 86)), ('▁presentar', (86, 96)), ('▁rector', (96, 103)), ('ra', (103, 105)), ('gia', (105, 108)), ('▁esca', (108, 113)), ('sa', (113, 115)), ('▁al', (115, 118)), ('▁final', (118, 124)), ('▁de', (124, 127)), ('▁la', (127, 130)), ('▁de', (130, 133)), ('posición', (133, 142)), ('▁junto', (142, 148)), ('▁con', (148, 152)), ('▁ten', (152, 156)), ('es', (156, 158)), ('mo', (158, 160)), ('▁y', (160, 162)), ('▁dolor', (162, 168)), ('▁anal', (168, 173)), ('▁de', (173, 176)), ('▁dos', (176, 180)), ('▁años', (180, 186)), ('▁de', (186, 189)), ('▁evolución', (189, 200)), ('.', (200, 201))]


[('▁La', (202, 204)), ('▁paciente', (2

In [70]:
check_id_frag = sum(train_frag[:check_id])
for frag in train_ind[check_id_frag:check_id_frag + train_frag[check_id]]:
    print(' '.join([tokenizer._convert_id_to_token(int(ind)) for ind in frag]), "\n")

<s> ▁Mu jer ▁de ▁42 ▁años ▁estudia da ▁en ▁Consulta s ▁de ▁Gastro enter ología ▁desde ▁enero ▁de ▁2005 ▁por ▁presentar ▁rector ra gia ▁esca sa ▁al ▁final ▁de ▁la ▁de posición ▁junto ▁con ▁ten es mo ▁y ▁dolor ▁anal ▁de ▁dos ▁años ▁de ▁evolución . </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> 

<s> ▁La ▁paciente ▁no ▁tenía ▁ningún ▁antecede nte ▁familiar ▁ni ▁personal ▁de ▁interés ▁ni ▁ segu ía ▁ningún ▁tratamiento ▁médico ▁de ▁forma ▁habitual . </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> 

In [71]:
# Fragment labels distribution
pd.Series(np.sum(train_y, axis=1)).describe()

count    7013.000000
mean        0.305290
std         0.652715
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         6.000000
dtype: float64

### Development corpus

Only development texts that are annotated with CIE-Procedimiento codes are considered:

In [72]:
# Some dev documents (texts) are not annotated 
len(set(df_text_dev["doc_id"]) - set(df_codes_dev_ner["doc_id"]))

28

In [73]:
dev_doc_list = sorted(set(df_codes_dev_ner["doc_id"]))

In [74]:
len(dev_doc_list)

222

In [75]:
%%time
ss_sub_corpus_path = ss_corpus_path + "dev/"
ss_files = [f for f in os.listdir(ss_sub_corpus_path) if os.path.isfile(ss_sub_corpus_path + f)]
ss_dict_dev = load_ss_files(ss_files, ss_sub_corpus_path)

CPU times: user 34.6 ms, sys: 77 µs, total: 34.7 ms
Wall time: 34.1 ms


In [76]:
%%time
dev_ind, dev_att, dev_y, dev_frag, dev_start_end_frag = ss_create_frag_input_data_xlmr(df_text=df_text_dev, 
                                                  text_col=text_col, 
                                                  df_ann=df_codes_dev_ner, doc_list=dev_doc_list, ss_dict=ss_dict_dev,
                                                  tokenizer=tokenizer, sp_pb2=spt, lab_encoder=mlb_encoder, seq_len=SEQ_LEN)

100%|██████████| 222/222 [00:02<00:00, 88.50it/s] 


CPU times: user 2.58 s, sys: 3.9 ms, total: 2.58 s
Wall time: 2.56 s


In [77]:
# Sanity check

In [78]:
dev_ind.shape

(3799, 128)

In [79]:
dev_att.shape

(3799, 128)

In [80]:
dev_y.shape

(3799, 727)

In [81]:
len(dev_frag)

222

In [82]:
len(dev_start_end_frag)

3799

In [83]:
# Check n_frag distribution across texts
pd.Series(dev_frag).describe()

count    222.000000
mean      17.112613
std        8.320553
min        4.000000
25%       11.000000
50%       15.000000
75%       21.000000
max       65.000000
dtype: float64

In [84]:
# Inspect a randomly selected text and its encoded version
check_id = np.random.randint(low=0, high=len(dev_doc_list), size=1)[0]

In [85]:
check_id

183

In [86]:
dev_doc_list[check_id]

'S1137-66272006000500011-1'

In [87]:
df_text_dev[df_text_dev["doc_id"] == dev_doc_list[check_id]][text_col].values[0]

'Varón de 46 años de edad, fumador y con alcoholismo crónico de aproximadamente 20 años de evolución, con la consiguiente hepatitis alcohólica diagnosticada hace ocho años. El paciente no presenta otros antecedentes de interés.\nNuestro paciente consulta por la presencia de masas en región cervical anterior y posterior, en zonas retroauriculares y en región antero-superior de tórax, de más de diez años de evolución. Estos bultomas no producen dolor, ni disnea ni disfagia.\nA la exploración se observa tumefacción en región cervical anterior, difusa, que borra la línea mandibular, junto a tumoraciones en zonas retroauriculares y en región deltoidea izquierda, más dos bultomas en región posterior de cuello que ascienden hasta la zona occipital, de unos 5 cm de diámetro cada uno. No son dolorosas a la palpación y son de consistencia dura.\n\nLa auscultación cardiopulmonar, la palpación abdominal y la exploración neurológica son normales.\nEn la analítica se observan los siguientes valores:

In [88]:
check_id_frag = sum(dev_frag[:check_id])
for i in range(check_id_frag, check_id_frag + dev_frag[check_id]):
    print(mlb_encoder.inverse_transform(np.array([dev_y[i]])), "\n")

[()] 

[()] 

[()] 

[()] 

[('0gbg',)] 

[()] 

[()] 

[()] 

[()] 

[('bw03zzz',)] 

[('0gbg', 'bg44zzz')] 

[('bw2f',)] 

[()] 

[()] 

[('0gbg',)] 

[()] 

[()] 



In [89]:
for i in range(check_id_frag, check_id_frag + dev_frag[check_id]):
    print(list(zip([tokenizer._convert_id_to_token(int(ind)) for ind in dev_ind[i]][1:len(dev_start_end_frag[i])+1], 
               dev_start_end_frag[i])))
    print("\n")

[('▁Var', (0, 3)), ('ón', (3, 6)), ('▁de', (6, 9)), ('▁46', (9, 12)), ('▁años', (12, 18)), ('▁de', (18, 21)), ('▁edad', (21, 26)), (',', (26, 27)), ('▁fum', (27, 31)), ('ador', (31, 35)), ('▁y', (35, 37)), ('▁con', (37, 41)), ('▁alcohol', (41, 49)), ('ismo', (49, 53)), ('▁cr', (53, 56)), ('ónico', (56, 62)), ('▁de', (62, 65)), ('▁aproximadamente', (65, 81)), ('▁20', (81, 84)), ('▁años', (84, 90)), ('▁de', (90, 93)), ('▁evolución', (93, 104)), (',', (104, 105)), ('▁con', (105, 109)), ('▁la', (109, 112)), ('▁con', (112, 116)), ('si', (116, 118)), ('gui', (118, 121)), ('ente', (121, 125)), ('▁hepatitis', (125, 135)), ('▁al', (135, 138)), ('co', (138, 140)), ('hó', (140, 143)), ('lica', (143, 147)), ('▁diagnostic', (147, 158)), ('ada', (158, 161)), ('▁hace', (161, 166)), ('▁och', (166, 170)), ('o', (170, 171)), ('▁años', (171, 177)), ('.', (177, 178))]


[('▁El', (179, 181)), ('▁paciente', (181, 190)), ('▁no', (190, 193)), ('▁presenta', (193, 202)), ('▁otros', (202, 208)), ('▁antecede', (2

In [90]:
check_id_frag = sum(dev_frag[:check_id])
for frag in dev_ind[check_id_frag:check_id_frag + dev_frag[check_id]]:
    print(' '.join([tokenizer._convert_id_to_token(int(ind)) for ind in frag]), "\n")

<s> ▁Var ón ▁de ▁46 ▁años ▁de ▁edad , ▁fum ador ▁y ▁con ▁alcohol ismo ▁cr ónico ▁de ▁aproximadamente ▁20 ▁años ▁de ▁evolución , ▁con ▁la ▁con si gui ente ▁hepatitis ▁al co hó lica ▁diagnostic ada ▁hace ▁och o ▁años . </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> 

<s> ▁El ▁paciente ▁no ▁presenta ▁otros ▁antecede ntes ▁de ▁interés . </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> 

In [91]:
# Fragment labels distribution
pd.Series(np.sum(dev_y, axis=1)).describe()

count    3799.000000
mean        0.309818
std         0.638081
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         5.000000
dtype: float64

### Training & Development corpus

We merge the previously generated datasets:

In [92]:
# Indices
train_dev_ind = np.concatenate((train_ind, dev_ind))

In [93]:
train_dev_ind.shape

(10812, 128)

In [94]:
# Attention masks
train_dev_att = np.concatenate((train_att, dev_att))

In [95]:
train_dev_att.shape

(10812, 128)

In [96]:
# y
train_dev_y = np.concatenate((train_y, dev_y))

In [97]:
train_dev_y.shape

(10812, 727)

## Fine-tuning

Using the corpus of labeled sentences, we fine-tune the model on a multi-label sentence classification task.

In [98]:
from transformers import TFXLMRobertaForSequenceClassification

model = TFXLMRobertaForSequenceClassification.from_pretrained(model_name, from_pt=True)

All PyTorch model weights were used when initializing TFXLMRobertaForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFXLMRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [99]:
model.summary()

Model: "tfxlm_roberta_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
roberta (TFRobertaMainLayer) multiple                  277453056 
_________________________________________________________________
classifier (TFRobertaClassif multiple                  592130    
Total params: 278,045,186
Trainable params: 278,045,186
Non-trainable params: 0
_________________________________________________________________


In [100]:
model.layers

In [101]:
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.initializers import GlorotUniform

input_ids = Input(shape=(SEQ_LEN,), name='input_ids', dtype='int64')
attention_mask = Input(shape=(SEQ_LEN,), name='attention_mask', dtype='int64')
inputs = [input_ids, attention_mask]

cls_token = model.layers[0](input_ids=inputs[0], attention_mask=inputs[1])[0][:, 0, :] # take <s> token output representation (equiv. to [CLS]) 
out_logits = Dense(units=num_labels, kernel_initializer=GlorotUniform(seed=random_seed))(cls_token) # Multi-label classification
out_act = Activation('sigmoid')(out_logits)

model = Model(inputs=[input_ids, attention_mask], outputs=out_act)

In [102]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 128)]        0                                            
__________________________________________________________________________________________________
attention_mask (InputLayer)     [(None, 128)]        0                                            
__________________________________________________________________________________________________
roberta (TFRobertaMainLayer)    TFBaseModelOutputWit 277453056   input_ids[0][0]                  
__________________________________________________________________________________________________
tf_op_layer_strided_slice (Tens [(None, 768)]        0           roberta[0][0]                    
______________________________________________________________________________________________

In [103]:
model.input

[<tf.Tensor 'input_ids:0' shape=(None, 128) dtype=int64>,
 <tf.Tensor 'attention_mask:0' shape=(None, 128) dtype=int64>]

In [104]:
model.output

<tf.Tensor 'activation_4/Identity:0' shape=(None, 727) dtype=float32>

In [ ]:
%%time
from tensorflow.keras import optimizers, losses
import tensorflow_addons as tfa

optimizer = tfa.optimizers.RectifiedAdam(learning_rate=LR)
loss = losses.BinaryCrossentropy(from_logits=False)
model.compile(optimizer=optimizer, loss=loss)

history = model.fit(x={'input_ids': train_dev_ind, 'attention_mask': train_dev_att}, y=train_dev_y,
          batch_size=BATCH_SIZE, epochs=EPOCHS, shuffle=True)

Epoch 1/41
676/676 [==============================] - 172s 255ms/step - loss: 0.2379
Epoch 2/41
676/676 [==============================] - 174s 257ms/step - loss: 0.0208
Epoch 3/41
676/676 [==============================] - 174s 257ms/step - loss: 0.0075
Epoch 4/41
676/676 [==============================] - 174s 257ms/step - loss: 0.0048
Epoch 5/41
676/676 [==============================] - 174s 257ms/step - loss: 0.0040
Epoch 6/41
676/676 [==============================] - 174s 257ms/step - loss: 0.0036
Epoch 7/41
676/676 [==============================] - 174s 258ms/step - loss: 0.0034
Epoch 8/41
676/676 [==============================] - 174s 258ms/step - loss: 0.0032
Epoch 9/41
676/676 [==============================] - 174s 258ms/step - loss: 0.0031
Epoch 10/41
676/676 [==============================] - 174s 258ms/step - loss: 0.0029
Epoch 11/41
676/676 [==============================] - 174s 258ms/step - loss: 0.0027
Epoch 12/41
676/676 [==============================] - 174s 258

## Test set predictions

Finally, the predictions made by the model on the test set are saved. For this purpose, firstly, each sentence from the test corpus must be converted into a sequence of subwords (input IDs and attention mask arrays). Then, the predictions made by the model at the sentence-level are saved, to be further evaluated at document-level (see `results/CodiEsp-P/Evaluation.ipynb`).

In [105]:
%%time
test_path = corpus_path + "test/text_files/"
test_files = [f for f in os.listdir(test_path) if os.path.isfile(test_path + f)]
test_data = load_text_files(test_files, test_path)
df_text_test = pd.DataFrame({'doc_id': [s.split('.txt')[0] for s in test_files], 'raw_text': test_data})

CPU times: user 2.67 ms, sys: 3.93 ms, total: 6.6 ms
Wall time: 6.07 ms


In [106]:
df_text_test.shape

(250, 2)

In [107]:
df_text_test.head()

,doc_id,raw_text
0,S1698-44472004000400012-1,"Varón de 54 años de edad, remitido a nuestro s..."
1,S1130-05582012000300005-1,Acude a nuestras consultas a un paciente que p...
2,S0212-16112009000300015-1,Se trató de un varón de 77 años con antecedent...
3,S1139-76322014000500014-1,Niño de cinco años derivado por su pediatra de...
4,S0212-71992004000300009-1,Varón de 22 años de edad que acude a consultas...


In [108]:
df_text_test.raw_text[0]

'Varón de 54 años de edad, remitido a nuestro servicio en mayo del 2003 por presentar odontalgia en relación con un tercer molar inferior derecho erupcionado. A la inspección oral se pudo observar la existencia de una tumefacción que expandía las corticales vestibulo-linguales, en la región del tercer molar mandibular derecho, cariado por distal. La mucosa oral estaba indemne, y no se palpaban adenomegalias cervicales. El paciente refería la existencia de una hipoestesia en el territorio de distribución del nervio mentoniano de quince días de evolución. En la ortopantomografía, se evidenció la presencia de una imagen radiolúcida, de contornos poco definidos, en el cuerpo mandibular derecho. Dos días después, bajo anestesia local, se procedió a la exodoncia del tercer molar y curetaje-biopsia del tejido subyacente. Durante el acto operatorio, se produjo una intensa hemorragia, que pudo ser cohibida con el empleo de Surgicel (Johnson & Johnson, Nuevo Brunswick, NJ) y mediante el empaquet

In [109]:
test_doc_list = sorted(set(df_text_test["doc_id"]))

In [110]:
len(test_doc_list)

250

In [111]:
%%time
ss_sub_corpus_path = ss_corpus_path + "test/"
ss_files = [f for f in os.listdir(ss_sub_corpus_path) if os.path.isfile(ss_sub_corpus_path + f)]
ss_dict_test = load_ss_files(ss_files, ss_sub_corpus_path)

CPU times: user 34.7 ms, sys: 0 ns, total: 34.7 ms
Wall time: 34.1 ms


In [112]:
%%time
test_ind, test_att, _, test_frag, _ = ss_create_frag_input_data_xlmr(df_text=df_text_test, 
                                                  text_col=text_col,
                                                  # Since labels are ignored, we pass df_codes_train_ner as df_ann
                                                  df_ann=df_codes_train_ner, doc_list=test_doc_list, ss_dict=ss_dict_test,
                                                  tokenizer=tokenizer, sp_pb2=spt, lab_encoder=mlb_encoder, seq_len=SEQ_LEN)

100%|██████████| 250/250 [00:00<00:00, 279.35it/s]


CPU times: user 956 ms, sys: 12.4 ms, total: 968 ms
Wall time: 951 ms


In [113]:
%%time
test_preds = model.predict({'input_ids': test_ind, 'attention_mask': test_att})

CPU times: user 14.9 s, sys: 1.68 s, total: 16.6 s
Wall time: 16.9 s


In [114]:
test_preds.shape

(3950, 727)

In [109]:
results_dir_path = "../results/CodiEsp-P/"

In [178]:
%%time
np.save(file=results_dir_path + "predictions/xlm_r_seed_" + str(random_seed) + "_test_preds.npy", arr=test_preds)

CPU times: user 2.21 ms, sys: 4.65 ms, total: 6.87 ms
Wall time: 6.02 ms


In [111]:
# To be further used when evaluating model performance at document level
np.save(file=results_dir_path + "xlm_r_test_frags.npy", arr=test_frag)
np.save(file=results_dir_path + "classes.npy", arr=mlb_encoder.classes_)